In [1]:
import pandas as pd
import ast 


In [2]:
# Load movies data on DF
df_movies = pd.read_csv('data/movies_dataset.csv', dtype={'popularity':object})

In [3]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [4]:
# Delete movies without title (Nill or empty)

df_movies.dropna(subset=['title'], inplace=True)

mask = df_movies['title'].str.len() > 1
df_movies = df_movies.loc[mask]


In [5]:
# Replace null with 0
df_movies['revenue'].fillna(0, inplace=True)

# Convert column to float & replace null with 0
df_movies.budget = df_movies.budget.astype(float)
df_movies['budget'].fillna(0, inplace=True)


In [6]:
# Extract collection name from belongs_to_collection string dict 

def collection_to_string(row):
    if type(row.belongs_to_collection) == str:
        collection = ast.literal_eval(row.belongs_to_collection)['name'] 
        return collection

    return None  

df_movies.belongs_to_collection = df_movies.apply(collection_to_string, axis=1)

In [7]:
# Extract countries names from production_countries string dict

def countries_to_list(row):
    if type(row.production_countries) == str:
        countries = ast.literal_eval(row.production_countries)
        list_countries = [c['name'] for c in countries]
        return str(list_countries)
    
    return '[]'

df_movies.production_countries = df_movies.apply(countries_to_list, axis=1)

In [8]:
# Extract companies names from production_companies string dict
def companies_to_list(row):

    if type(row.production_companies) == str:
        companies = ast.literal_eval(row.production_companies)
        list_companies = [c['name'] for c in companies]
        return str(list_companies)
    
    return '[]'

df_movies.production_companies = df_movies.apply(companies_to_list, axis=1)

In [9]:
# Extract gen names from genres string dict
def genres_to_list(row):

    if type(row.genres) == str:
        genero = ast.literal_eval(row.genres)
        if len(genero) >0:
            list_genero = [g['name'] for g in genero]
            return str(list_genero)
    
    return '[]'

df_movies.genres = df_movies.apply(genres_to_list, axis=1)

In [10]:
# Delete rows with empty release date
df_movies.count()
df_movies.dropna(subset=['release_date'], inplace=True)

In [11]:
# Add release year column
df_movies['release_year'] = df_movies['release_date'].apply(lambda x : int(x[:4]))

In [12]:
# Calculate return

def calculo_return (row):
    if type(row.revenue)==float and type(row.budget)==float and row.budget>0:
        return round(row.revenue/row.budget, 2)
    return 0

df_movies['return'] = df_movies.apply(calculo_return, axis=1)


In [13]:
# add director from credits
def extract_director(row):

    if type(row.crew) == str:
        crew = ast.literal_eval(row.crew)
        
        if len(crew)> 0:
            for d in crew:
                if 'Director' in d.values():
                    director = d['name']
                    return director
    
    return ''

df_credits = pd.read_csv('data/credits.csv')
df_credits['director'] = df_credits.apply(extract_director, axis=1)
df_credits.drop(columns=['crew','cast'], inplace=True)

#make ids same type
df_movies.id = df_movies.id.astype(int)
#add director column to df_movies
df_movies = pd.merge(df_movies, df_credits, on='id', how='inner')

In [14]:
#make overiew str for all rows
df_movies['overview'].fillna('', inplace=True)

In [15]:
#drop duplicates titles
df_movies  = df_movies.drop_duplicates(subset='title', keep='last')

In [16]:
#Drop useless columns
df_movies.drop(columns=['video','imdb_id','adult','original_title','poster_path'\
                        ,'homepage', 'spoken_languages', 'tagline', 'video'], inplace=True)

In [17]:
#Save clean DataFrame
df_movies.to_parquet('data/movies_clean.pq')
del df_movies
del df_credits